In [ ]:
# random sampling
import rss
import numpy as np
import time
start_time = time.time()
    
parameters = {}
net_list = []
net_list.append({'net_name':'KNN','sizes':[256,256],'dim_cor':[64,64],'mode':'tensor','weights':'distance'})

parameters['net_p'] = {'gpu_id':'cpu','net_name':'composition','net_list':net_list}
parameters['data_p'] = {'data_shape':(256,256),'random_rate':0.5,
                        'pre_full':True,'mask_type':'random','down_sample_rate':2,
                        'data_type':'gray_img','data_path':'./data/img/baboon.bmp'}
parameters['train_p'] = {'train_epoch':20}
parameters['show_p'] = {'show_type':'gray_img','show_content':'original'}
parameters['opt_p'] = {'net': {'opt_name': 'Adam', 'lr': 1e-1, 'weight_decay': 0}}
rssnet = rss.rssnet(parameters)

rssnet.show()
rssnet.show_p['show_content'] = 'recovered'
rssnet.net.net_list[0].update_neighbor(n_neighbors=8,mask=rssnet.mask.cpu())
for i in range(10):
    rssnet.train()
    if i % 4 == 0:
        rssnet.net.net_list[0].update_neighbor(n_neighbors=8,mask=rssnet.mask.cpu(),mode='patch',labda=1,n_components=8)
print(time.time()-start_time)
rssnet.show()

In [ ]:
# Tucker for random sampling
import rss
import numpy as np

    
parameters = {}
net_list = []
net_list.append({'net_name':'KNN','sizes':[256,256],'dim_cor':[256,256],'mode':'tensor'})

parameters['net_p'] = {'gpu_id':0,'net_name':'composition','net_list':net_list}
parameters['data_p'] = {'data_shape':(256,256),'random_rate':0.5,
                        'pre_full':True,'mask_type':'patch','down_sample_rate':2,
                        'data_type':'gray_img','data_path':'./data/img/man.bmp'}
parameters['train_p'] = {'train_epoch':10000}
parameters['show_p'] = {'show_type':'gray_img','show_content':'original'}
rssnet = rss.rssnet(parameters)

rssnet.show()
rssnet.show_p['show_content'] = 'recovered'
rssnet.net.net_list[0].update_neighbor(n_neighbors=5,mask=rssnet.mask.cpu())
for i in range(10):
    rssnet.train()
    rssnet.show()
    

In [ ]:
# TF->INR
import rss

parameters = {}
hash_para = {'net_name':'HashEmbedder','finest_resolution':512,'n_levels':2,'n_features_per_level':1,'base_resolution':512}
hash_para = {'net_name':'HashEmbedder','finest_resolution':256,'n_levels':16,'n_features_per_level':2,'base_resolution':16}
parameters['net_p'] = {'gpu_id':0,'net_name':'HashINR','hash_mode':'vanilla',
                       'hash_para':hash_para,
                       'inr_para':{'net_name':'MLP','num_layers':1,'dim_hidden':16}}

parameters['data_p'] = {'data_shape':(256,256),'random_rate':0.5,
                        'pre_full':False,'mask_type':'img','mask_path':'../mask/mask.png',
                        'data_path':'../img/Cameraman.jpg','data_type':'gray_img','ymode':'completion',
                       'noise_mode':'gaussian','noise_parameter':25}

parameters['train_p'] = {'train_epoch':100}

parameters['show_p'] = {'show_type':'gray_img','show_content':'original'}

parameters['opt_p'] = {'net': {'opt_name': 'Adam', 'lr': 1e-1, 'weight_decay': 0},
                       'reg': {'opt_name': 'Adam', 'lr': 1e-3, 'weight_decay': 0},
                       'noise': {'opt_name': 'Adam', 'lr': 1e-4, 'weight_decay': 0}}
huber_reg = {'reg_name':'MultiReg','reg_list':[{'reg_name':'INRR','coef':1e-2,'n':256,'mode':0,
                                                'w0_initial':1.,'lap_k':1,'lap_mode':'Huber',
                                                'huber_delta':0.2,'inrr_alpha':1,
                                                'nabla_matrix_order_k':1,
                                               'inr_parameter':{'dim_in': 1,'dim_out':100,'w0_initial':10}},
                                              {'reg_name':'INRR','coef':1e-2,'n':256,'mode':1,
                                               'w0_initial':1.,'lap_k':1,'lap_mode':'Huber',
                                               'huber_delta':0.2,'inrr_alpha':1,
                                               'nabla_matrix_order_k':1,
                                              'inr_parameter':{'dim_in': 1,'dim_out':100,'w0_initial':10}}]}
air_reg = {'reg_name':'MultiReg','reg_list':[{'reg_name':'AIR','coef':1e-3,'n':256,'mode':0,'lap_mode':'Huber'},
                                             {'reg_name':'AIR','coef':1e-3,'n':256,'mode':1,'lap_mode':'Huber'}]}
parameters['reg_p'] = huber_reg
rssnet = rss.rssnet(parameters)

rssnet.show()
rssnet.show_p['show_content'] = 'recovered'
for i in range(10):
    rssnet.train()
    rssnet.show()
 